In [1]:
# importing needed libraries for the project

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import re
import json
import tweepy 
import datetime
import seaborn as sns 
import requests
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Gathering Data 

### There are 3 main sources of Data 
- main archive
- image predictions 
- Tiwitter itself for the full tweets details (api)

In [2]:
# Downloading Image predictions file 

url = ' https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = url.split('/')[-1]

if not os.path.isfile(file_name):
    response = requests.get(url)
    with open(file_name, 'wb') as f:
        f.write(response.content)

In [3]:
#loading main archive df  
main_df = pd.read_csv('twitter-archive-enhanced.csv')

main_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [4]:
#loardin image predictions 
img_df = pd.read_csv('image-predictions.tsv', sep='\t')

In [5]:
#assising image pred
img_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [6]:
# tweepy api details 

api_key = ********************************
api_secret_key = ********************************
bearer_token = ********************************
access_token = ********************************
access_secret = ********************************


In [7]:
# get the tweets archive using api and the tweets id from the main archive 

auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True , wait_on_rate_limit_notify=True)

In [8]:
# testing the code on getting one tweet
# also this will help us take a look on the tweet structure and content
temp_tweet = api.get_status(main_df['tweet_id'][1000], tweet_mode = 'extended')
content = temp_tweet._json
print(content)

{'created_at': 'Wed Jun 29 01:23:16 +0000 2016', 'id': 747963614829678593, 'id_str': '747963614829678593', 'full_text': 'PUPPER NOOOOO BEHIND YOUUU 10/10 pls keep this pupper in your thoughts https://t.co/ZPfeRtOX0Q', 'truncated': False, 'display_text_range': [0, 70], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 747963600220917761, 'id_str': '747963600220917761', 'indices': [71, 94], 'media_url': 'http://pbs.twimg.com/media/CmFM7ngXEAEitfh.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CmFM7ngXEAEitfh.jpg', 'url': 'https://t.co/ZPfeRtOX0Q', 'display_url': 'pic.twitter.com/ZPfeRtOX0Q', 'expanded_url': 'https://twitter.com/dog_rates/status/747963614829678593/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 937, 'h': 632, 'resize': 'fit'}, 'small': {'w': 680, 'h': 459, 'resize': 'fit'}, 'large': {'w': 937, 'h': 632, 'resize': 'fit'}}}]}, 'extended_entities': {'media': [{'id': 74796

In [9]:
#so this was hard to view so let's have a better view 
for k, v in content.items():
    print(k)
    print(v)
    print('----------')

created_at
Wed Jun 29 01:23:16 +0000 2016
----------
id
747963614829678593
----------
id_str
747963614829678593
----------
full_text
PUPPER NOOOOO BEHIND YOUUU 10/10 pls keep this pupper in your thoughts https://t.co/ZPfeRtOX0Q
----------
truncated
False
----------
display_text_range
[0, 70]
----------
entities
{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 747963600220917761, 'id_str': '747963600220917761', 'indices': [71, 94], 'media_url': 'http://pbs.twimg.com/media/CmFM7ngXEAEitfh.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CmFM7ngXEAEitfh.jpg', 'url': 'https://t.co/ZPfeRtOX0Q', 'display_url': 'pic.twitter.com/ZPfeRtOX0Q', 'expanded_url': 'https://twitter.com/dog_rates/status/747963614829678593/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 937, 'h': 632, 'resize': 'fit'}, 'small': {'w': 680, 'h': 459, 'resize': 'fit'}, 'large': {'w': 937, 'h': 632, 'resize': 'fit'}}}]}
----------
ex

In [10]:
#Wanted to have a better closer look at the user and entities dict 
print('user dict:')
for k, v in content['user'].items():
    print(k)
    print(v)
    print('----------')    
print('\nentities dict:')
for k, v in content['entities'].items():
    print(k)
    print(v)
    print('----------')    
    

user dict:
id
4196983835
----------
id_str
4196983835
----------
name
WeRateDogs®
----------
screen_name
dog_rates
----------
location
「 DM YOUR DOGS 」
----------
description
Your Only Source For Professional Dog Ratings Instagram and Facebook ➪ WeRateDogs partnerships@weratedogs.com ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
----------
url
https://t.co/Wrvtpnv7JV
----------
entities
{'url': {'urls': [{'url': 'https://t.co/Wrvtpnv7JV', 'expanded_url': 'https://blacklivesmatters.carrd.co', 'display_url': 'blacklivesmatters.carrd.co', 'indices': [0, 23]}]}, 'description': {'urls': []}}
----------
protected
False
----------
followers_count
8850586
----------
friends_count
17
----------
listed_count
5863
----------
created_at
Sun Nov 15 21:41:29 +0000 2015
----------
favourites_count
145858
----------
utc_offset
None
----------
time_zone
None
----------
geo_enabled
True
----------
verified
True
----------
statuses_count
12783
----------
lang
None
----------
contributors_enabled
False
----------
is_translator
False
----

In [11]:
#getting the full tweets data through the api
#there are 2 expected outputs here, json for data collected, and error tweets in a variable and file 
errors = []
count = 0
if not os.path.isfile('full_tweets_json.txt'):
    with open('full_tweets_json.txt', 'w') as f:
        for tweet_id in main_df['tweet_id']:
            try:
                tweet = api.get_status(tweet_id, tweet_mode = 'extended', wait_on_rate_limit =True, 
                                       wait_on_rate_limit_notify = True)
                json.dump(tweet._json,f)
                f.write('\n')
                count += 1
                print('Success on tweet_id :{}'.format(tweet_id))
            except Exception as e:
                print(('Error: {}, on tweet_id :{}').format(str(e), tweet_id))
                error = {}
                error['tweet_id'] = tweet_id
                error['error_type'] = str(e)
                errors.append(error)
    erros_df = pd.DataFrame(errors)
    #create a fil for error tweets, might need to compare their id 
    if not os.path.isfile('error_tweets.csv'):
        erros_df.to_csv('error_tweets.csv', index=False)
    print('Total Successful tweets aquired : {}'.format(count))
    print('Total Error tweets : {}'.format(len(errors)))
    print(erros_df.error_type.value_counts())

In [12]:
# loading the data we need from each tweet
#according to the test tweet exploration, I decided to get (favorite_count, retweet_count, followers_count, hashtags)
#tweet followers indcate user followers at that tweet, it most probaly increases between tweets
#we have to get the tweet_id also to be able to join on later


api_data = []

with open('full_tweets_json.txt', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweet_id = tweet['id_str']
        tweet_retweets = tweet['retweet_count']
        tweet_favorites = tweet['favorite_count']
        tweet_followers_count = tweet['user']['followers_count']
        tweet_hashtags = tweet['entities']['hashtags']
        api_data.append({'tweet_id': tweet_id,
                     'tweet_retweet_count' : tweet_retweets,
                     'tweet_facorite_count' : tweet_favorites,
                     'followers_count' : tweet_followers_count,
                     'hashtags': tweet_hashtags})

api_df = pd.DataFrame(api_data)

# Assessing 

### Visual Assesment 

In [13]:
#assisng main archive visually using info - head - sample 

main_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1081,738885046782832640,NaN,NaN,2016-06-04 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charles. He's a Nova Scotian Towel Pou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/738885046...,11,10,Charles,None,None,None,None
893,759446261539934208,NaN,NaN,2016-07-30 17:51:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",No no no this is all wrong. The Walmart had to...,NaN,NaN,NaN,https://twitter.com/wsaznews/status/7591675587...,10,10,None,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None
1166,722613351520608256,NaN,NaN,2016-04-20 02:30:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Schnitzel. He's a Tropicana Floofboop. Ge...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722613351...,12,10,Schnitzel,None,None,None,None
1776,677961670166224897,NaN,NaN,2015-12-18 21:20:32 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Izzy. She's showing off the dance move...,NaN,NaN,NaN,https://vine.co/v/iKuMDuYV0aZ,11,10,Izzy,None,None,None,None


### Notes: 
- a lot of NaN values in:retweets columns, but this is expected, need to check it programaticly to check signfigance, maybe used to remove retwweets 
- as mentioned int the project brief the rating numerator is higher than denominator
- there are 4 columns (doggo, floofer, pupper, puppo) that could be melted, have to check programticly ot confirm
- values in the 4 columns mentioned are either the same name of the column or none
- found some names consisting of one letter 

In [14]:
#assisng image predictions visually 
img_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
16,666102155909144576,https://pbs.twimg.com/media/CT54YGiWUAEZnoK.jpg,1,English_setter,0.298617,True,Newfoundland,0.149842,True,borzoi,0.133649,True
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False,tow_truck,0.029175,False,shopping_cart,0.026321,False
49,666835007768551424,https://pbs.twimg.com/media/CUES51dXIAEahyG.jpg,1,Airedale,0.448459,True,toy_poodle,0.124030,True,teddy,0.110183,False
1025,710658690886586372,https://pbs.twimg.com/media/CdzETn4W4AAVU5N.jpg,1,soft-coated_wheaten_terrier,0.948617,True,Dandie_Dinmont,0.018664,True,cairn,0.015943,True
1723,819924195358416896,https://pbs.twimg.com/ext_tw_video_thumb/81992...,1,bathtub,0.100896,False,shower_curtain,0.091866,False,tub,0.049176,False


## Notes:
- column names are not the descriptive (p1, p1_conf, p1_dog, etc)
- prediction values some times starts with capitlaized letter and some times not
- the false predictions are really of topic, can we know how did the module got those ?



In [15]:
# assising api df visually 

api_df.sample(5)

,tweet_id,tweet_retweet_count,tweet_facorite_count,followers_count,hashtags
198,852672615818899456,2065,14425,8855782,[]
1202,713919462244790272,775,3225,8855786,[]
1955,672975131468300288,848,1589,8855801,[]
1169,717537687239008257,1805,5651,8855786,[]
1624,683773439333797890,1316,3285,8855791,[]


## Notes
- hashtags column showing empty list instead of none, when there's value it's a list of dict
- followers count changing from time to time can be used as phases in the account growth though the campaign 
- number of retweets and favorites can be used to get a real rating system for dogs 

## Programtic assesment 

In [16]:
# for the main archive lets start by checking the names' issues 
not_names = ['a','all','my','his','the','an','O','None','one' ,' by', "Al", 'old']
main_df[(main_df.name.str.len()<4)&(~main_df.name.isin(not_names))].name.value_counts()
#main_df[main_df.name == 'Al']

Bo     9
Leo    6
Jax    6
Gus    5
Mia    3
Ted    3
Max    3
Doc    2
Tyr    2
Moe    2
Eli    2
Ava    2
Eve    2
not    2
Ken    2
Bob    2
mad    2
Ash    2
Sam    2
Lou    2
Ben    1
JD     1
Edd    1
Zoe    1
Blu    1
Pip    1
Dug    1
Stu    1
Jay    1
Jim    1
Obi    1
Sky    1
Amy    1
Taz    1
Evy    1
Rey    1
Mo     1
Cal    1
Jo     1
Sid    1
Aja    1
Ike    1
Ace    1
Dex    1
Dot    1
Ron    1
Jeb    1
Tom    1
Alf    1
Mya    1
Gin    1
Jed    1
Ito    1
Ed     1
by     1
Ole    1
Tug    1
Mac    1
Name: name, dtype: int64

In [17]:
# witll check the text of tweets with odd names from the api df
pd.options.display.max_colwidth = 200
main_df[main_df.name == 'an'].text



759     RT @dog_rates: This is an East African Chalupa Seal. We only rate dogs. Please only send in dogs. Thank you... 10/10 https://t.co/iHe6liLwWR
1025      This is an Iraqi Speed Kangaroo. It is not a dog. Please only send in dogs. I'm very angry with all of you ...9/10 https://t.co/5qpBTTpgUt
1362                   This is an East African Chalupa Seal. We only rate dogs. Please only send in dogs. Thank you... 10/10 https://t.co/iHe6liLwWR
2204     This is an Irish Rigatoni terrier named Berta. Completely made of rope. No eyes. Quite large. Loves to dance. 10/10 https://t.co/EM5fDykrJg
2333     This is an extremely rare horned Parthenon. Not amused. Wears shoes. Overall very nice. 9/10 would pet aggressively https://t.co/QpRjllzWAL
2335        This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv
2349      This is an odd dog. Hard on the outside but loving on the inside. Petting still fun. Doesn't pla

In [18]:
#checking classification columns 
classes = ['doggo','floofer', 'pupper', 'puppo']
for c in classes:
    rem_classes = classes.copy()
    num_c = len(main_df[main_df[c] != 'None'])
    print('number of {} classification is : {}'.format(c,num_c))
    rem_classes.remove(c)
    for r in rem_classes:
        intersection = main_df[(main_df[c] != 'None') & (main_df[r] != 'None')]
        num_i = len(intersection)
        print('number of double classification {} and {} is : {}'.format(c,r,num_i))




number of doggo classification is : 97
number of double classification doggo and floofer is : 1
number of double classification doggo and pupper is : 12
number of double classification doggo and puppo is : 1
number of floofer classification is : 10
number of double classification floofer and doggo is : 1
number of double classification floofer and pupper is : 0
number of double classification floofer and puppo is : 0
number of pupper classification is : 257
number of double classification pupper and doggo is : 12
number of double classification pupper and floofer is : 0
number of double classification pupper and puppo is : 0
number of puppo classification is : 30
number of double classification puppo and doggo is : 1
number of double classification puppo and floofer is : 0
number of double classification puppo and pupper is : 0


In [ ]:
main_clean.rating_numerator.value_counts()

In [ ]:
main_clean.rating_denominator.value_counts()

In [19]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

### Notes:
##### regarding names:
- wrong values totaly, the algorithm missed the name and took the word before it 
- None
- part of name like "O'Mally" coming as "O" , "Al Cabone" coming "Al"
- wrong names are ; ['a','all','my','his','the','an','O','None','one' ,' by', "Al"]

##### Regarding Classification:
- not all dogs are classified 
- the most populer class is floofer
- doggo class itersect once with floofer and puppo, and 12 times with pupper 

##### Regarding data to be dropped:
- there are 2 columns that excludes retweets which are ouf of scope, so will remove 78 points 
- the extended columns includes tweets with images which identify points to be kept 

##### Regarding rating:
- rating denminator could be changed to the max value in numerator so we can we have a better rating system
- but the values of both rating_denominator and rating_numerator or both inaccurate and out of proptotions 

##### Data size:
- we have total of 2356, from which 2297 have images and there's 78 retweets to be romved 
- we need to filter on these points first before connecting to other DFs

##### Data Types:
- tweet_id is int needs to be converted to str 
- retweet columns are float, if we were to use them chould have changed them to str 
- the id of retweets of our original tweets are in floats so needs to be also converted to str (retweet_status_id, retweet_status_user_id)
- timestamp is loaded as str, needs to be onverted to datetime format 

In [20]:
img_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [21]:
#image archive assesment 
img_df.p3_dog.value_counts()

True     1499
False     576
Name: p3_dog, dtype: int64

##### Data Size:
- we have 2075 image in total 
- only tweet_id to filter so no retweet markers here

##### Column Names:
- as in visual assessment, it could be changed to be more descriptive 

##### values:
- number of unique values predicted are p1:378 ,p2:405 ,p3:1499
- number of true predictions : p1:1532 ,p2:1553 ,p3:408
- most predicted bread: p1:golden_retriever ,p2:Labrador_retriever ,p3:Labrador_retriever    
- most predicted false value: p1:seat_belt ,p2:seat_belt ,p3:doormat

##### Valid predictions ?:
- the predictions are evaluated on 2 metrics, level of machine confidence, is the prediction a dog type or not 
- there's not confirming if it predict the correct breed, just that it predicted "a" dog breed 
- there should be a refrence if we want ot confirm by adding the breed either from owners text - given they know the dog- or from another pet expert which would be better but we don't have it 
- we could try checking tect for dog breeds to confirm

In [22]:
#assessing api_df 
api_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tweet_id              2331 non-null   object
 1   tweet_retweet_count   2331 non-null   int64 
 2   tweet_facorite_count  2331 non-null   int64 
 3   followers_count       2331 non-null   int64 
 4   hashtags              2331 non-null   object
dtypes: int64(3), object(2)
memory usage: 91.2+ KB


In [23]:
#exploring the hashtag to check if it's significant or not 
len(api_df[api_df.hashtags.str.len() != 0])

27

In [24]:
api_df.followers_count.min()

8855781

##### Column names:
- first 3 columns are headed with the word "tweet_", which is not nesccessery come to think about it, except for the first ofc 
- either we load the data again from the file or just remove it from the 2nd and 3rd column 

##### Hashtags:
- number of tweets of hasthags is 27 out of 2331 we have in this df 
- number is insgnificant in analysis so we can drop it totally 
- my personal descion is to clean it and keep it 

##### Values:
- most retweeted got: 76270, least got : 1
- most favorited tweet got : 153910 , least got : 0 
- highest followers count: 8856459 , lowest: 8855781 , the change or increase in followers count isn't huge through these many tweets ?!

## Final Assessment Report: 

###  Main_df:
##### regarding names:
- wrong values totaly, the algorithm missed the name and took the word before it 
- None
- part of name like "O'Mally" coming as "O" , "Al Cabone" coming "Al"
- wrong names are ; ['a','all','my','his','the','an','O','None','one' ,' by', "Al"]

##### Regarding Classification:
- not all dogs are classified 
- values in the 4 columns mentioned are either the same name of the column or none
- the most populer class is floofer
- doggo class itersect once with floofer and puppo, and 12 times with pupper 

##### Regarding data to be dropped:
- there are 2 columns that excludes retweets which are ouf of scope, so will remove 78 points 
- the extended columns includes tweets with images which identify points to be kept 

##### Regarding rating:
- as mentioned int the project brief the rating numerator is almost always higher than denominator
- rating denminator could be changed to the max value in numerator so we can we have a better rating system

##### Data size:
- we have total of 2356, from which 2297 have images and 78 retweets
- we need to filter on these points first before connecting to other DFs

##### Data Types:
- tweet_id is int needs to be converted to str 
- retweet columns are float, if we were to use them chould have changed them to str 
- the id of retweets of our original tweets are in floats so needs to be also converted to str (retweet_status_id, retweet_status_user_id)
- timestamp is loaded as str, needs to be onverted to datetime format

### img_df:
##### Data Size:
- we have 2075 image in total 
- only tweet_id to filter so no retweet markers here

##### Column Names:
- column names are not that descriptive (p1, p1_conf, p1_dog, etc)

##### values:
- prediction values some times starts with capitlaized letter and some times not
- number of unique values predicted are p1:378 ,p2:405 ,p3:1499
- number of true predictions : p1:1532 ,p2:1553 ,p3:408
- most predicted bread: p1:golden_retriever ,p2:Labrador_retriever ,p3:Labrador_retriever    
- most predicted false value: p1:seat_belt ,p2:seat_belt ,p3:doormat

##### Valid predictions ?:
- the predictions are evaluated on 2 metrics, level of machine confidence, is the prediction a dog type or not 
- there's not confirming if it predict the correct breed, just that it predicted "a" dog breed 
- there should be a refrence if we want ot confirm by adding the breed either from owners text - given they know the dog- or from another pet expert which would be better but we don't have it 
- we could try checking tect for dog breeds to confirm

### api_df:
##### Column names:
- first 3 columns are headed with the word "tweet_", which is not nesccessery come to think about it, except for the first ofc 
- either we load the data again from the file or just remove it from the 2nd and 3rd column 

##### Hashtags:
- number of tweets of hasthags is 27 out of 2331 we have in this df 
- hashtags column showing empty list instead of none, when there's value it's a list of dict
- number is insgnificant in analysis so we can drop it totally 
- my personal descion is to clean it and keep it 

##### Values:
- number of retweets and favorites can be used to get a real rating system for dogs 
- most retweeted got: 76270, least got : 1
- most favorited tweet got : 153910 , least got : 0 
- highest followers count: 8856459 , lowest: 8855781 , the change or increase in followers count isn't huge through these many tweets ?!



# Assesment Summary:

## Quality issues:
### main_df:
- incomplete data; in name columns 
- inacurate data; wrong entries in name column due to wrong extraction as (a ,an, the, old, by ,...)
- incomplete data in classification columns
- inconsistant represtnation of missing values in the table ,'None' -- NaN
- inacurate data in rating columns numerator almost always higher than denominator
- values in tweet_id needs to be changed to str 
- values in timestamp needs to be convereted to datetime

### img_df:
- inacurate data regarding true classification, it only says if the prediction is a breed or not, not if it's the right bread
- we could argue that the p_dog is only to make sure that the prediction is a breed so it achieved it's purpose then

### api_df:
- incomplete data in hashtags column, there is no entries
- incositant values of hashtages, needs to be either NaN or strings

## Tidenees issues:
- length of the 3 dfs not matching
- in main_df there is 4 columns with value names as header, and can be joined into one column
- in img_df the column needs discrpetive names 


# Cleaning

In [25]:
# first will create copies of the dfs 

main_clean = main_df.copy()
img_clean = img_df.copy()
api_clean = api_df.copy()

## Define:
### Main_df:
- starting by the clearest step, droping retweets,replies (non originals) from the(in_replay_to, retweet_id) columns
- then drop tweets with no imgs, using the (extended_url) as refrence
- compare to the length of img df (after removing retweets from it ), as both should have same size
- change values of tweet_id
- change values of timestamp
- fix the rating system by getting max numerator and set as denominator, then setting denominator to 10 again and get the factor to apply to all numerator column 
- will create a new class for dogs with 2 classification and remove their values from the orignal columns 
- this will provide 6 columns of classifcations to be melted together in one column with the name (dog_class)

##### Dog Names:
- as much as I'm intreged to fix dog names, it's a reptitve process that can't be programticlly fixed for all with one code, as the text extractor already missed it before and there's no creiteria for Names as word to selct upon, even capital letters are used in other non-name words, so texts has to be assesed visually 
- the statistical signifigance is low for the wrong names, given the many 'None' Values
- will re-assess again after the previously mentioned points, as I'm intreged to do it 

## Code

In [26]:
main_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [27]:
# dropping retweets and replies
# dropping tweets with no expanded urls --> no imgs

main_clean = main_clean[main_clean.in_reply_to_status_id.isnull()]
main_clean = main_clean[main_clean.retweeted_status_id.isnull()]
main_clean = main_clean[main_clean.expanded_urls.notnull()]

# now drop the columns itsef
main_clean = main_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1)

In [28]:
# changing tweet_id value from int to str
# changing timestamp to datetime format 
main_clean.tweet_id = main_clean.tweet_id.astype(str)
main_clean['timestamp'] = pd.to_datetime(main_clean['timestamp'])

##### after removing non original tweets from img_clean compare 2 dfs, should match 

In [29]:
# fixing the classes issue
# starting by creating new columns for double classes 
# we know from assesing that only doggo class that intersect with others
# will create new classes for intersections


other_classes = ['floofer','pupper','puppo']
for x in other_classes:
    condition = ((main_clean['doggo'] != 'None') & (main_clean[x] != 'None'))
    main_clean['{}-doggo'.format(x)] = np.where(condition, '{}-doggo'.format(x), 'None')



In [30]:
# now we remove values in original columns if it exists in the intersection culmns 

new_classes = main_clean.columns[-3:]

for n in new_classes:
    old_class = n[:-6]
    main_clean.loc[main_clean[n] != 'None','doggo'] = 'None'
    main_clean.loc[main_clean[n] != 'None',old_class] = 'None'


In [42]:
# I will create a no_class column to be helpful in melting 

condition = ((main_clean['doggo'] == 'None') & (main_clean['floofer'] == 'None')&
            (main_clean['pupper'] == 'None') & (main_clean['puppo'] == 'None') &
            (main_clean['floofer-doggo'] == 'None') & (main_clean['pupper-doggo'] == 'None') &
            (main_clean['puppo-doggo'] == 'None'))
main_clean['not_stage'] = np.where(condition, 'not_stage','None')

In [ ]:
# this becomes irrlivant after cleaning
print(len(main_clean[main_clean.doggo != 'None']))
print(len(main_clean[main_clean.floofer != 'None']))
print(len(main_clean[main_clean.pupper != 'None']))
print(len(main_clean[main_clean.puppo != 'None']))
print(len(main_clean[main_clean['floofer-doggo'] != 'None']))
print(len(main_clean[main_clean['pupper-doggo'] != 'None']))
print(len(main_clean[main_clean['puppo-doggo'] != 'None']))
print(len(main_clean[main_clean['not_stage'] != 'None']))

In [32]:
#main_clean.replace(to_replace=['None'], value=['-'], inplace= True )

In [45]:
# now we gather them into one column 
# start by melting  the 4 columns into one colamn called dog_class
# data size should be maultiplied by 8 
main_clean.columns[:-7]

main_clean = pd.melt(main_clean, id_vars= ('tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
                                           'rating_numerator', 'rating_denominator', 'name'), 
                     var_name='dog_class', value_name='dog_stage')

# starting by changing 'None' to NaN
#main_clean.replace(to_replace=['None'], value= np.nan, inplace= True )



In [53]:
# now we remove extra rows 
stages = ['doggo', 'floofer', 'pupper', 'puppo', 'floofer-doggo','pupper-doggo', 'puppo-doggo','not_stage']
main_clean = main_clean[main_clean.dog_stage.isin(stages)]

# and now we drop the extra column we added (value and variable are the same)
main_clean.drop(['dog_class'], axis=1, inplace=True)

In [136]:
# My on examining rating system on a closer look, I decided to drop it totally
# the values are out of propotions totaly and need to be fixed manually by checking text of tweets
# even then the statistical signifigance of it is almost none
main_clean.drop(['rating_numerator', 'rating_denominator'], axis=1, inplace=True)

## Testing

In [137]:
main_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 9 to 16751
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   tweet_id       1964 non-null   object             
 1   timestamp      1964 non-null   datetime64[ns, UTC]
 2   source         1964 non-null   object             
 3   text           1964 non-null   object             
 4   expanded_urls  1964 non-null   object             
 5   name           1964 non-null   object             
 6   dog_stage      1964 non-null   object             
dtypes: datetime64[ns, UTC](1), object(6)
memory usage: 122.8+ KB


In [55]:
main_clean.dog_stage.value_counts()

not_stage        1759
pupper            220
doggo              72
puppo              23
pupper-doggo        9
floofer             9
puppo-doggo         1
floofer-doggo       1
Name: dog_stage, dtype: int64

### Define

#### img_df:
- change the column names to be more descriptive 
- change tweet_id to str
- check imd_clean.tweet_id against main_clean to remove retweets and replies 
- check main_clean.tweet_id agains img_clean to remove tweets without rated predicted imgs

In [69]:
img_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               2075 non-null   object 
 1   jpg_url                2075 non-null   object 
 2   img_num                2075 non-null   int64  
 3   first_predect          2075 non-null   object 
 4   first_confidence_lvl   2075 non-null   float64
 5   first_predict_is_dog   2075 non-null   bool   
 6   second_predect         2075 non-null   object 
 7   second_confidence_lvl  2075 non-null   float64
 8   second_predict_is_dog  2075 non-null   bool   
 9   third_predect          2075 non-null   object 
 10  third_confidence_lvl   2075 non-null   float64
 11  third_predict_is_dog   2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


### Code

In [64]:
img_clean.tweet_id = img_clean.tweet_id.astype(str)

In [68]:
img_clean.rename(columns={'p1':'first_predect', 'p1_conf':'first_confidence_lvl', 'p1_dog': 'first_predict_is_dog',
                          'p2':'second_predect', 'p2_conf':'second_confidence_lvl', 'p2_dog': 'second_predict_is_dog',
                          'p3':'third_predect', 'p3_conf':'third_confidence_lvl', 'p3_dog': 'third_predict_is_dog'},
                inplace=True)

In [70]:
main_tweets = main_clean.tweet_id

In [71]:
main_tweets

9        890240255349198849
39       884162670584377345
86       872967104147763200
94       871515927908634625
95       871102520638267392
                ...        
16747    666049248165822465
16748    666044226329800704
16749    666033412701032449
16750    666029285002620928
16751    666020888022790149
Name: tweet_id, Length: 2094, dtype: object

In [73]:
len(img_clean[img_clean.tweet_id.isin(main_tweets)])

1971

In [74]:
# removing img tweets that are not in main_clean 
img_clean = img_clean[img_clean.tweet_id.isin(main_tweets)]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               1971 non-null   object 
 1   jpg_url                1971 non-null   object 
 2   img_num                1971 non-null   int64  
 3   first_predect          1971 non-null   object 
 4   first_confidence_lvl   1971 non-null   float64
 5   first_predict_is_dog   1971 non-null   bool   
 6   second_predect         1971 non-null   object 
 7   second_confidence_lvl  1971 non-null   float64
 8   second_predict_is_dog  1971 non-null   bool   
 9   third_predect          1971 non-null   object 
 10  third_confidence_lvl   1971 non-null   float64
 11  third_predict_is_dog   1971 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 159.8+ KB


In [76]:
# now we remove from the main_clean data 
clean_img_tweets = img_clean.tweet_id
main_clean = main_clean[main_clean.tweet_id.isin(clean_img_tweets)]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 9 to 16751
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1971 non-null   object             
 1   timestamp           1971 non-null   datetime64[ns, UTC]
 2   source              1971 non-null   object             
 3   text                1971 non-null   object             
 4   expanded_urls       1971 non-null   object             
 5   rating_numerator    1971 non-null   int64              
 6   rating_denominator  1971 non-null   int64              
 7   name                1971 non-null   object             
 8   dog_stage           1971 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 154.0+ KB


### Testing

In [78]:
img_clean.info()
main_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               1971 non-null   object 
 1   jpg_url                1971 non-null   object 
 2   img_num                1971 non-null   int64  
 3   first_predect          1971 non-null   object 
 4   first_confidence_lvl   1971 non-null   float64
 5   first_predict_is_dog   1971 non-null   bool   
 6   second_predect         1971 non-null   object 
 7   second_confidence_lvl  1971 non-null   float64
 8   second_predict_is_dog  1971 non-null   bool   
 9   third_predect          1971 non-null   object 
 10  third_confidence_lvl   1971 non-null   float64
 11  third_predict_is_dog   1971 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 159.8+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 9 to 16751
Data columns (tot

### Define:
- fix the hashtags column 
- check tweet_id between api and main


### Code

In [115]:
api_clean = api_df.copy()

In [116]:
api_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tweet_id              2331 non-null   object
 1   tweet_retweet_count   2331 non-null   int64 
 2   tweet_facorite_count  2331 non-null   int64 
 3   followers_count       2331 non-null   int64 
 4   hashtags              2331 non-null   object
dtypes: int64(3), object(2)
memory usage: 91.2+ KB


In [90]:
#get a look on the hashtags column
api_df[api_df.hashtags.str.len() != 0]

,tweet_id,tweet_retweet_count,tweet_facorite_count,followers_count,hashtags
4,891327558926688256,8332,37256,8855781,"[{'text': 'BarkWeek', 'indices': [129, 138]}]"
5,891087950875897856,2794,18780,8855781,"[{'text': 'BarkWeek', 'indices': [129, 138]}]"
8,890609185150312448,3848,25834,8855781,"[{'text': 'BarkWeek', 'indices': [113, 122]}]"
10,890006608113172480,6559,28429,8855781,"[{'text': 'BarkWeek', 'indices': [121, 130]}]"
14,889531135344209921,2022,14052,8855781,"[{'text': 'BarkWeek', 'indices': [109, 118]}]"
15,889278841981685760,4776,23335,8855781,"[{'text': 'BarkWeek', 'indices': [129, 138]}]"
31,886054160059072513,99,0,8855781,"[{'text': 'BATP', 'indices': [21, 26]}]"
56,881268444196462592,4739,21546,8855781,"[{'text': 'Canada150', 'indices': [111, 121]}]"
70,878776093423087618,3676,17970,8855781,"[{'text': 'PrideMonthPuppo', 'indices': [29, 45]}, {'text': 'PrideMonth', 'indices': [115, 126]}]"
93,874012996292530176,9224,31858,8855781,"[{'text': 'PrideMonth', 'indices': [122, 133]}]"


In [117]:
#create a hastag esxtractor
# if we found tweets with more hashtags would have added more features, this is designed for 1 or 2 hashtags 

def extract_hashtag(x):
    if len(x) != 0:
        if len(x) ==2 :
            text1 = x[0]['text']
            text2 = x[1]['text']
            text = text1 + ' - ' + text2
        else:
            text = x[0]['text']
    else:
        text='no_hashtag'
    return text

In [118]:
api_clean.hashtags = api_clean.hashtags.apply(extract_hashtag)


In [146]:
# remove tweet_ from begining retweet and favorite comlumns 
api_clean = api_clean.rename(columns ={'tweet_retweet_count':'retweet_count', 'tweet_facorite_count': 'facorite_count'})

In [128]:
# match tweet_id 
main_clean_tweets = main_clean.tweet_id
main_clean_tweets

9        890240255349198849
39       884162670584377345
86       872967104147763200
94       871515927908634625
104      869596645499047938
                ...        
16747    666049248165822465
16748    666044226329800704
16749    666033412701032449
16750    666029285002620928
16751    666020888022790149
Name: tweet_id, Length: 1964, dtype: object

In [120]:
api_clean = api_clean[api_clean.tweet_id.isin(main_clean_tweets)]

In [121]:
api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2330
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tweet_id              1964 non-null   object
 1   tweet_retweet_count   1964 non-null   int64 
 2   tweet_facorite_count  1964 non-null   int64 
 3   followers_count       1964 non-null   int64 
 4   hashtags              1964 non-null   object
dtypes: int64(3), object(2)
memory usage: 92.1+ KB


In [123]:
api_clean_tweets = api_clean.tweet_id
api_clean_tweets 

0       892420643555336193
1       892177421306343426
2       891815181378084864
3       891689557279858688
4       891327558926688256
               ...        
2326    666049248165822465
2327    666044226329800704
2328    666033412701032449
2329    666029285002620928
2330    666020888022790149
Name: tweet_id, Length: 1964, dtype: object

In [124]:
main_clean = main_clean[main_clean.tweet_id.isin(api_clean_tweets)]

In [125]:
main_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 9 to 16751
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1964 non-null   object             
 1   timestamp           1964 non-null   datetime64[ns, UTC]
 2   source              1964 non-null   object             
 3   text                1964 non-null   object             
 4   expanded_urls       1964 non-null   object             
 5   rating_numerator    1964 non-null   int64              
 6   rating_denominator  1964 non-null   int64              
 7   name                1964 non-null   object             
 8   dog_stage           1964 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 153.4+ KB


In [129]:
main_clean_tweets = main_clean.tweet_id
main_clean_tweets

9        890240255349198849
39       884162670584377345
86       872967104147763200
94       871515927908634625
104      869596645499047938
                ...        
16747    666049248165822465
16748    666044226329800704
16749    666033412701032449
16750    666029285002620928
16751    666020888022790149
Name: tweet_id, Length: 1964, dtype: object

In [130]:
# now let's bring img_clean df to the same ids 

img_clean = img_clean[img_clean.tweet_id.isin(main_clean_tweets)]
img_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               1964 non-null   object 
 1   jpg_url                1964 non-null   object 
 2   img_num                1964 non-null   int64  
 3   first_predect          1964 non-null   object 
 4   first_confidence_lvl   1964 non-null   float64
 5   first_predict_is_dog   1964 non-null   bool   
 6   second_predect         1964 non-null   object 
 7   second_confidence_lvl  1964 non-null   float64
 8   second_predict_is_dog  1964 non-null   bool   
 9   third_predect          1964 non-null   object 
 10  third_confidence_lvl   1964 non-null   float64
 11  third_predict_is_dog   1964 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 159.2+ KB


### Test

In [135]:
# now all three df should have same values in tweet_id and same size
api_clean.info()
main_clean.info()
img_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2330
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tweet_id              1964 non-null   object
 1   tweet_retweet_count   1964 non-null   int64 
 2   tweet_facorite_count  1964 non-null   int64 
 3   followers_count       1964 non-null   int64 
 4   hashtags              1964 non-null   object
dtypes: int64(3), object(2)
memory usage: 92.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 9 to 16751
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1964 non-null   object             
 1   timestamp           1964 non-null   datetime64[ns, UTC]
 2   source              1964 non-null   object             
 3   text                1964 non-null   object             
 4   expanded_urls  

In [122]:
api_clean.hashtags.value_counts()

no_hashtag                      1942
BarkWeek                           8
PrideMonth                         3
dogsatpollingstations              1
NoDaysOff - swole                  1
notallpuppers                      1
FinalFur                           1
PrideMonthPuppo - PrideMonth       1
ScienceMarch                       1
WKCDogShow                         1
WomensMarch                        1
K9VeteransDay                      1
BellLetsTalk                       1
Canada150                          1
Name: hashtags, dtype: int64

# Storing

In [144]:
# first we merge the three dfs on one main_clean_df
main_clean_df = pd.merge(main_clean, img_clean, on = 'tweet_id', how= 'outer')
main_clean_df = main_clean_df.merge(api_clean, on ='tweet_id', how= 'outer')

In [147]:
main_clean_df = main_clean_df.rename(columns ={'tweet_retweet_count':'retweet_count', 'tweet_facorite_count': 'facorite_count'})

In [148]:
main_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 1963
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1964 non-null   object             
 1   timestamp              1964 non-null   datetime64[ns, UTC]
 2   source                 1964 non-null   object             
 3   text                   1964 non-null   object             
 4   expanded_urls          1964 non-null   object             
 5   name                   1964 non-null   object             
 6   dog_stage              1964 non-null   object             
 7   jpg_url                1964 non-null   object             
 8   img_num                1964 non-null   int64              
 9   first_predect          1964 non-null   object             
 10  first_confidence_lvl   1964 non-null   float64            
 11  first_predict_is_dog   1964 non-null   bool             

In [141]:
main_clean_df.tail()

,tweet_id,timestamp,source,text,expanded_urls,name,dog_stage,jpg_url,img_num,first_predect,first_confidence_lvl,first_predict_is_dog,second_predect,second_confidence_lvl,second_predict_is_dog,third_predect,third_confidence_lvl,third_predict_is_dog
1959,666049248165822465,2015-11-16 00:24:50+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,https://twitter.com/dog_rates/status/666049248165822465/photo/1,None,not_stage,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
1960,666044226329800704,2015-11-16 00:04:52+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,https://twitter.com/dog_rates/status/666044226329800704/photo/1,a,not_stage,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
1961,666033412701032449,2015-11-15 23:21:54+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,https://twitter.com/dog_rates/status/666033412701032449/photo/1,a,not_stage,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
1962,666029285002620928,2015-11-15 23:05:30+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,https://twitter.com/dog_rates/status/666029285002620928/photo/1,a,not_stage,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
1963,666020888022790149,2015-11-15 22:32:08+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj,https://twitter.com/dog_rates/status/666020888022790149/photo/1,None,not_stage,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True


In [149]:
# save data to a csv file 
main_clean_df.to_csv('twitter_archive_master.csv', index= False)